<a href="https://colab.research.google.com/github/rahul-bavaliya/saskjobs/blob/main/SaskJobs_Scrapping.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Importing Requests and Beautifulsoup ready to use.

In [ ]:
import requests
from bs4 import BeautifulSoup
import time
import warnings
import pandas as pd
import matplotlib.pyplot as plt
import random
import re
import string
import datetime

Fetching IMDb films urls and loop them to access data between 2010 and 2020.

In [ ]:
# IMDb URL to get the data for a year
BASE_URL = "https://www.saskjobs.ca/jsp/joborder/listing.jsp?filter_by=2&noc_skill_type=2&region_id=14&region_id=11&page={page}"


global jobs_count
jobs_count = 0

global total_pages
total_pages = 0

global SASK_JOB_URI
SASK_JOB_URI = "https://www.saskjobs.ca"

In [ ]:
#Custom Exceptions
# 1. Custom BadResponse from the serer.
class BadResponseException(Exception):
   def __init__(self, arg,response):
       self.msg=arg

class MoreRequestException(Exception):
  def __init__(self, message, errors):
        # Call the base class constructor with the parameters it needs
        super().__init__(message)

        # Now for your custom code...
        self.errors = errors

Gather the all Jobs data and parse with beautiful soup.

In [ ]:
def fetch_data(page_no):
  job_position = None
  job_employer = None
  job_location = None
  job_posting_date = None
  job_openings = None
  global jobs_count
  global total_pages
  global SASK_JOB_URI
  try:

    # Build URL with current year and starting position.
    SASK_JOBS_URI = BASE_URL.format(page = page_no)

    # Make GET request to current URL.
    jobs_page = requests.get(SASK_JOBS_URI)

    # Check if request was successful.
    if jobs_page.status_code == 200:

      # Parse HTML content with BeautifulSoup.
      soup = BeautifulSoup(jobs_page.content, "html.parser")

      # Find list of movie containers.
      div_category_bottom = soup.find("div", {"id": "category_bottom"})
      # print(div_category_bottom)
      tr_jobs_table_bottom = div_category_bottom.find('table')("tr")[1]("td")[0]("table")[0]("tr")

      for tr_job_table_bottom in tr_jobs_table_bottom[2:-2]:
        job_position_hyperLink_job = tr_job_table_bottom('td')[0]('a', href= True)
        job_position_noc_code = tr_job_table_bottom('td')[0]('span')[0].string
        if job_position_hyperLink_job:
          job_position = job_position_hyperLink_job[0].text



        job_employer_td = tr_job_table_bottom('td')[1]
        if job_employer_td:
          job_employer = job_employer_td.text

        job_location_td = tr_job_table_bottom('td')[2]
        if job_location_td:
          job_location = job_location_td.text

        job_posting_date_span = tr_job_table_bottom('td')[3]('span')[0]
        if job_posting_date_span:
          job_posting_date_text = job_posting_date_span.text
          job_posting_date=datetime.datetime.strptime(job_posting_date_text,"%d-%b-%Y")

        job_openings_td = tr_job_table_bottom('td')[4]
        if job_openings_td:
          job_openings = job_openings_td.text

        # Add movie information to dataframe.
        print(
            "Page_no:",page_no,'|',
            job_position_noc_code,'|',
           f'{job_position}','|',
            str(job_employer),'|',
            str(job_location),'|',
            job_posting_date,'|',
            int(job_openings),'|',
            SASK_JOB_URI+job_position_hyperLink_job[0]['href'],
            )

        jobs_df.loc[jobs_count] = [
            str(job_position),
            job_position_noc_code,
            str(job_employer),
            str(job_location),
            job_posting_date,
            int(job_openings),
            '=HYPERLINK("{}")'.format(SASK_JOB_URI+job_position_hyperLink_job[0]['href']),
            ]
        jobs_count += 1
        total_pages = len(tr_jobs_table_bottom[-1]('td')[1].findAll('a', href=True))
      page_no += 1
      if total_pages>= page_no: fetch_data(page_no)



    else:
      jobs_count = 0
      raise HTTPError("Internal Server Error.")

    # tr[last()]

  except HTTPError as httpError:
    print(httpError.args[0])

  return jobs_df

In [ ]:
from requests.models import HTTPError

# Empty list to hold movie data.
jobs_df = pd.DataFrame(columns=("Position",'NOC_Code',"Employer","Location","Date","Job_Openings","Link"))

#Page Number
page_no = 1

# Request count: To count each request for the API call.
request_count = 0

# Count the number of jobs scraped
jobs_count = 0

jobs_df = fetch_data(page_no)

jobs_df.sort_values(by = ["Employer"], axis=0, ascending=True,inplace=True, na_position='first')

jobs_df.to_csv('Data_Jobs_Regina_Saskatoon.csv')

Page_no: 1 | NOC 2171 | Financial Admin Business Systems Analyst | City of Regina | REGINA | 2023-08-18 00:00:00 | 1 | https://www.saskjobs.ca/jsp/joborder/detail.jsp?job_order_id=1267685
Page_no: 1 | NOC 2171 | Enterprise Architect - Information Technology (It) | Finney Taylor Consulting Group | REGINA | 2023-08-18 00:00:00 | 1 | https://www.saskjobs.ca/jsp/joborder/detail.jsp?job_order_id=1267637
Page_no: 1 | NOC 2174 | Application and Solutions Developer (Junior) | AGT Foods and Ingredients Inc. | REGINA | 2023-08-18 00:00:00 | 1 | https://www.saskjobs.ca/jsp/joborder/detail.jsp?job_order_id=1267606
Page_no: 1 | NOC 2253 | Assistant Production Technician | REGENCY GAMING INDUSTRIES | SASKATOON | 2023-08-18 00:00:00 | 1 | https://www.saskjobs.ca/jsp/joborder/detail.jsp?job_order_id=1267595
Page_no: 1 | NOC 2174 | Junior Software Developer | Mega Group Inc. / Mega Groupe Inc. | SASKATOON | 2023-08-18 00:00:00 | 1 | https://www.saskjobs.ca/jsp/joborder/detail.jsp?job_order_id=1267571
P